In [6]:
from gurobipy import *
from itertools import combinations, product

In [7]:
m = 3
n = 3

arcs = []
for layer in range(m-1):
    for i in range(n):
        for j in range(n):
            arcs.append((n*layer+i+1 , n*(layer+1)+j+1))

print(arcs)

[(1, 4), (1, 5), (1, 6), (2, 4), (2, 5), (2, 6), (3, 4), (3, 5), (3, 6), (4, 7), (4, 8), (4, 9), (5, 7), (5, 8), (5, 9), (6, 7), (6, 8), (6, 9)]


In [8]:
def testsys(not_shortest):
    LP = Model()
    LP.Params.LogToConsole = 0
    x = LP.addVars(arcs, vtype=GRB.CONTINUOUS, name="x")
    for a in arcs:
        LP.addConstr(x[a] >= 0)
    for i in range(n):
        for j in range(n):
            for m1 in range(n):
                for m2 in range(m1 + 1, n):
                    a = i + 1
                    c = 2 * n + j + 1
                    b1 = n + m1 + 1
                    b2 = n + m2 + 1
                    if (a, b1, c) in not_shortest and (a, b2, c) in not_shortest:
                        continue;
                    elif (a, b1, c) in not_shortest:
                        LP.addConstr(x[(a, b2)] + x[(b2, c)] + 1 <= x[(a, b1)] + x[(b1, c)])
                    elif (a, b2, c) in not_shortest:
                        LP.addConstr(x[(a, b1)] + x[(b1, c)] + 1 <= x[(a, b2)] + x[(b2, c)])
                    else:
                        LP.addConstr(x[(a, b1)] + x[(b1, c)] == x[(a, b2)] + x[(b2, c)])
    LP.setObjective(quicksum(x[a] for a in arcs))
    LP.optimize()
    #obj = LP.getObjective()
    return LP.status


layer1 = [i + 1 for i in range(n)]
layer2 = [n + i + 1 for  i in range(n)]
layer3 = [2*n + i + 1 for i in range(n)]
t1 = list(combinations(layer1, 2))
t2 = list(combinations(layer2, 2))
t3 = list(combinations(layer3, 2))
# t1 = [(1, i) for i in layer1[1:]]
# t2 = [(1+n, i) for i in layer2[1:]]
# t3 = [(1+2*n, i) for i in layer3[1:]]
subgraphst = list(product(t1, t2, t3))
subgraphs = []
for t in subgraphst:
    tlist = t[0]+t[1]+t[2]
    subgraphs.append(tlist)
print(subgraphs)

def detect_forbidden_minor(notshortest):
    for s in subgraphs:
        contained = []
        for p in notshortest:
            if set(p).issubset(set(s)):
                contained.append(p)
        if len(contained) == 1:
            return [1, s]
        elif len(contained) <= 4 and len(contained) >= 2:
            pairs = list(combinations(contained, 2))
            forbidden = True
            for p in pairs:
                overlap = set(p[0]).intersection(set(p[1]))
                if len(overlap) != 1:
                    forbidden = False
                    break
            if forbidden:
                return [2, s]
    return [0, []]


[(1, 2, 4, 5, 7, 8), (1, 2, 4, 5, 7, 9), (1, 2, 4, 5, 8, 9), (1, 2, 4, 6, 7, 8), (1, 2, 4, 6, 7, 9), (1, 2, 4, 6, 8, 9), (1, 2, 5, 6, 7, 8), (1, 2, 5, 6, 7, 9), (1, 2, 5, 6, 8, 9), (1, 3, 4, 5, 7, 8), (1, 3, 4, 5, 7, 9), (1, 3, 4, 5, 8, 9), (1, 3, 4, 6, 7, 8), (1, 3, 4, 6, 7, 9), (1, 3, 4, 6, 8, 9), (1, 3, 5, 6, 7, 8), (1, 3, 5, 6, 7, 9), (1, 3, 5, 6, 8, 9), (2, 3, 4, 5, 7, 8), (2, 3, 4, 5, 7, 9), (2, 3, 4, 5, 8, 9), (2, 3, 4, 6, 7, 8), (2, 3, 4, 6, 7, 9), (2, 3, 4, 6, 8, 9), (2, 3, 5, 6, 7, 8), (2, 3, 5, 6, 7, 9), (2, 3, 5, 6, 8, 9)]


In [9]:
allpaths = []

for i in range(n):
    for j in range(n):
        for k in range(n):
            a = i + 1
            b = n + j + 1
            c = 2 * n + k + 1
            allpaths.append((a, b, c))
print(allpaths)

l = len(allpaths)

[(1, 4, 7), (1, 4, 8), (1, 4, 9), (1, 5, 7), (1, 5, 8), (1, 5, 9), (1, 6, 7), (1, 6, 8), (1, 6, 9), (2, 4, 7), (2, 4, 8), (2, 4, 9), (2, 5, 7), (2, 5, 8), (2, 5, 9), (2, 6, 7), (2, 6, 8), (2, 6, 9), (3, 4, 7), (3, 4, 8), (3, 4, 9), (3, 5, 7), (3, 5, 8), (3, 5, 9), (3, 6, 7), (3, 6, 8), (3, 6, 9)]


In [11]:
minimal = []

fixed = (1, 1+n, 1+2*n)
for a in range(1, 10):
    allpathsys = list(combinations(range(1, l), a))
    for s in allpathsys:
        notshortest = [fixed]
        is_minimal = True
        for i in s:
            notshortest.append(allpaths[i])
        for m in minimal:
            if set(m).issubset(set(notshortest)):
                is_minimal = False
                break
        if not is_minimal:
            continue
        out = detect_forbidden_minor(notshortest)
        if out[0] == 0:
            if testsys(notshortest) == 2:
                print(notshortest)
                minimal.append(notshortest)
            else:
                print("Invalid minimal System containing no forbidden minors:")
                print(notshortest)

[(1, 4, 7), (1, 4, 8), (1, 4, 9)]
[(1, 4, 7), (1, 5, 7), (1, 6, 7)]
[(1, 4, 7), (2, 4, 7), (3, 4, 7)]
[(1, 4, 7), (1, 4, 8), (1, 5, 7), (1, 5, 8), (2, 6, 9), (3, 6, 9)]
[(1, 4, 7), (1, 4, 8), (1, 6, 7), (1, 6, 8), (2, 5, 9), (3, 5, 9)]
[(1, 4, 7), (1, 4, 8), (2, 4, 7), (2, 4, 8), (3, 5, 9), (3, 6, 9)]
[(1, 4, 7), (1, 4, 8), (2, 5, 9), (2, 6, 9), (3, 4, 7), (3, 4, 8)]
[(1, 4, 7), (1, 4, 8), (2, 5, 9), (2, 6, 9), (3, 5, 9), (3, 6, 9)]
[(1, 4, 7), (1, 4, 9), (1, 5, 7), (1, 5, 9), (2, 6, 8), (3, 6, 8)]
[(1, 4, 7), (1, 4, 9), (1, 6, 7), (1, 6, 9), (2, 5, 8), (3, 5, 8)]
[(1, 4, 7), (1, 4, 9), (2, 4, 7), (2, 4, 9), (3, 5, 8), (3, 6, 8)]
[(1, 4, 7), (1, 4, 9), (2, 5, 8), (2, 6, 8), (3, 4, 7), (3, 4, 9)]
[(1, 4, 7), (1, 4, 9), (2, 5, 8), (2, 6, 8), (3, 5, 8), (3, 6, 8)]
[(1, 4, 7), (1, 5, 7), (2, 4, 7), (2, 5, 7), (3, 6, 8), (3, 6, 9)]
[(1, 4, 7), (1, 5, 7), (2, 6, 8), (2, 6, 9), (3, 4, 7), (3, 5, 7)]
[(1, 4, 7), (1, 5, 7), (2, 6, 8), (2, 6, 9), (3, 6, 8), (3, 6, 9)]
[(1, 4, 7), (1, 6, 7), (2, 

KeyboardInterrupt: 